Importing the necessary modules

In [8]:
import numpy as np
import pandas as pd
from obspy import read
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import os

In [9]:
from sklearn.ensemble import RandomForestClassifier

In [10]:
from obspy.signal.invsim import cosine_taper
from obspy.signal.filter import highpass
from obspy.signal.trigger import classic_sta_lta, plot_trigger, trigger_onset

In [11]:
from scipy.fft import fft, fftfreq

In [12]:
import joblib

Function to preprocess input and Predict

In [53]:
def Predict_type(minseed,csv):
    st = read(minseed)
    # This is how you get the data and the time, which is in seconds
    tr = st.traces[0].copy()
    tr_data = tr.data
    df = tr.stats.sampling_rate
    sta_len = 120
    lta_len = 600
    cft = classic_sta_lta(tr_data, int(sta_len * df), int(lta_len * df))
    thr_on = 4
    thr_off = 1.5
    on_off = np.array(trigger_onset(cft, thr_on, thr_off))
    # How long should the short-term and long-term window be, in seconds?
    # Run Obspy's STA/LTA to obtain a characteristic function
    # This function basically calculates the ratio of amplitude between the short-term 
    # and long-term windows, moving consecutively in time across the data
    data = pd.read_csv(csv)
    data=data.loc[data['time_rel(sec)']>=(on_off[0][0])]
    time = data['time_rel(sec)']  # Assuming the first column is time
    amplitude = data['velocity(m/s)']  # Assuming the second column is amplitude
    amplitude=np.asfarray(amplitude)
    # Number of sample points
    N = len(amplitude)
    # Sample spacing (assuming uniform sampling)
    time=np.array(time)
    T = time[1] - time[0]
    # Perform the Fourier Transform
    yf = 2.0/N * np.abs(fft(amplitude)[:N//2])
    xf = fftfreq(N, T)[:N//2]
    max_velocity=np.max(yf)
    weighted_velocity=np.sum(yf*xf)/np.sum(xf)
    mean_velocity=np.mean(yf)
    freq_of_max=xf[np.argmax(yf)]
    area=(np.sum(yf*xf))
    arr= np.array([max_velocity, weighted_velocity, mean_velocity, freq_of_max, area])
    min_arr=np.array([5.500321581195321e-12, 1.1357021950539033e-13,2.352262677275252e-13, 0.108138752105393, 1.4515146530837311e-08])
    max_arr=np.array([1.2684432046057191e-09,5.205067987341181e-11,9.485446966158655e-11,0.9731615173627284,2.0500911085770924e-06])
    arr= (arr-min_arr)/(max_arr-min_arr)
    clf_loaded = joblib.load('moonquake_model.joblib')
    x= clf_loaded.predict([arr])[0]
    if x==1:
        return "impact_mq"
    elif x==2:
        return "deep_mq"
    elif x==3:
        return "shallow_mq"

In [ ]:
Predict_type("Testit.mseed","Testit.csv")